# Выбор локации для скважины

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Нужно построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

План действий:

После загрузки и подготовки данных, обучу модель для каждого региона, оценю RMSE этой модели. Затем проанализирую результаты и проведу расчет прибыли для каждого региона. Рассчитаю достаточный объем сырья для безубыточной разработки новой скважины. Сравню полученный объем сырья со средним запасом в каждом регионе. Далее буду работать со скважиной с максимальными значениями предсказаний. Рассчитаю прибыль для суммы объема сырья, соответствующего предсказаниям. Посчитаю риски и прибыль для каждого региона (Bootstraр, чтобы найти распределение прибыли.ю и найду среднюю прибыль, 95%-й доверительный интервал и риск убытков. После анализа и вывода о проделанной работе можно предложить регион для разработки скважин. 

## Импорт и изучение данных:

In [5]:
# Импорт библиотек:
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState

In [3]:
# Импорт данных:
data_0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
data_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
data_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')

Изучим данные:

In [8]:
data_0.info(), data_1.info(), data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

(None, None, None)

In [9]:
data_0.head(), data_1.head(), data_2.head()

(      id        f0        f1        f2     product
 0  txEyH  0.705745 -0.497823  1.221170  105.280062
 1  2acmU  1.334711 -0.340164  4.365080   73.037750
 2  409Wp  1.022732  0.151990  1.419926   85.265647
 3  iJLyR -0.032172  0.139033  2.978566  168.620776
 4  Xdl7t  1.988431  0.155413  4.751769  154.036647,
       id         f0         f1        f2     product
 0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
 1  62mP7  14.272088  -3.475083  0.999183   26.953261
 2  vyE1P   6.263187  -5.948386  5.001160  134.766305
 3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
 4  AHL4O  12.702195  -8.147433  5.004363  134.766305,
       id        f0        f1        f2     product
 0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
 1  WJtFt  0.262778  0.269839 -2.530187   56.069697
 2  ovLUW  0.194587  0.289035 -5.586433   62.871910
 3  q6cA6  2.236060 -0.553760  0.930038  114.572842
 4  WPMUX -0.515993  1.716266  5.899011  149.600746)

Вывод: Во всех 3 data по 100 тысяч строк, 5 колонок, а тип данных float64, кроме id, что есть норма. Пропусков в данных нет. Данные не нуждаются в подготовке. 

## Обучение и проверка модели:

2.1 Разобьем данные на обучающую и валидационную выборки в соотношении 75:25:

In [10]:
#Разделю дата на целевой признак и остальное:
target_0 = data_0['product']
features_0 = data_0.drop(['product', 'id'], axis=1)

In [11]:
#Разобью данные на обучающую и валидационную выборки:
features_train_0, features_valid_0, target_train_0,target_valid_0 = \
train_test_split(features_0, target_0, test_size=0.25, random_state=12345)

Проделаю этот этап отдельно для каждого региона:

In [12]:
#Разделю дата на целевой признак и остальное:
target_1 = data_1['product']
features_1 = data_1.drop(['product', 'id'], axis=1)
#Разобью данные на обучающую и валидационную выборки:
features_train_1, features_valid_1, target_train_1,target_valid_1 = \
train_test_split(features_1, target_1, test_size=0.25, random_state=12345)

In [13]:
#Разделю дата на целевой признак и остальное:
target_2 = data_2['product']
features_2 = data_2.drop(['product', 'id'], axis=1)
#Разобью данные на обучающую и валидационную выборки:
features_train_2, features_valid_2, target_train_2,target_valid_2 = \
train_test_split(features_2, target_2, test_size=0.25, random_state=12345)

2.2. Обучение модели и предсказание на валидационной выборке:

In [14]:
#Обучу модель:
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
#Предсказания на валидационной выборке:
predicted_valid_0 = model_0.predict(features_valid_0)

In [15]:
#Обучу модель:
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
#Предсказания на валидационной выборке:
predicted_valid_1 = model_1.predict(features_valid_1)

In [16]:
#Обучу модель:
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
#Предсказания на валидационной выборке:
predicted_valid_2 = model_2.predict(features_valid_2)

2.3. Посчитаем средний запас предсказанного сырья и RMSE модели:

In [17]:
# посчитаю rmse:
rmse_0 = mean_squared_error(target_valid_0, predicted_valid_0)**0.5
    
# посчитаю средний запас сырья:
predicted_valid_mean_0 = predicted_valid_0.sum()/len(predicted_valid_0)

In [18]:
# посчитаю rmse:
rmse_1 = mean_squared_error(target_valid_1, predicted_valid_1)**0.5
    
# посчитаю средний запас сырья:
predicted_valid_mean_1 = predicted_valid_1.sum()/len(predicted_valid_1)

In [19]:
# посчитаю rmse:
rmse_2 = mean_squared_error(target_valid_2, predicted_valid_2)**0.5
    
# посчитаю средний запас сырья:
predicted_valid_mean_2 = predicted_valid_2.sum()/len(predicted_valid_2)

In [20]:
#Напечатаю результат вместе:
print('Средний запас сырья в Регионе_0 =', predicted_valid_mean_0, 'rmse =', rmse_0)
print('Средний запас сырья в Регионе_1 =', predicted_valid_mean_1, 'rmse =', rmse_1)
print('Средний запас сырья в Регионе_2 =', predicted_valid_mean_2, 'rmse =', rmse_2)

Средний запас сырья в Регионе_0 = 92.59256778438035 rmse = 37.5794217150813
Средний запас сырья в Регионе_1 = 68.72854689544602 rmse = 0.8930992867756165
Средний запас сырья в Регионе_2 = 94.96504596800489 rmse = 40.02970873393434


Вывод: Средних запасов сырья больше всего в регионе_2, rmse так же выше всего в регионе_2.

## Подготовка к расчёту прибыли:

3.1. Все ключевые значения для расчётов сохраним в отдельных переменных.

In [22]:
#Создам отдельные переменные для ключевых значения:
#Бюджет на 200 скважин:
budget_200 = 10*10**9
#500 исследуемых очек:
points_500 = 500 
#Отобранные 200 лучших точек:
points_200 = 200
budget_point = budget_200 / points_200
#Доход:
profit = 450
#Доход с каждого продукта:
profit_1 = 450 * 10**3
#Порог вероятности убытков:
threshold = 2,5 

3.2. Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе.

In [23]:
#Объем сырья для безубыточной разработки скважин:
volume_without_losses = budget_200 / points_200 / profit

In [24]:
#Поделим на 1000, округлим до 3 цифр:
print(f'Достаточный объём сырья для безубыточной разработки новой скважины = {round(volume_without_losses/1000, 3)}')

Достаточный объём сырья для безубыточной разработки новой скважины = 111.111


In [25]:
print('Разница среднего запаса сырья в Регионе_0 и достаточного объема сырья равна', \
      data_0['product'].mean()*1000 - volume_without_losses)

Разница среднего запаса сырья в Регионе_0 и достаточного объема сырья равна -18611.111111111095


In [26]:
print('Разница среднего запаса сырья в Регионе_1 и достаточного объема сырья равна', \
      data_1['product'].mean()*1000 - volume_without_losses)

Разница среднего запаса сырья в Регионе_1 и достаточного объема сырья равна -42286.111111111095


In [27]:
print('Разница среднего запаса сырья в Регионе_2 и достаточного объема сырья равна', \
      data_2['product'].mean()*1000 - volume_without_losses)

Разница среднего запаса сырья в Регионе_2 и достаточного объема сырья равна -16111.111111111066


Вывод: 
По разнице среднего запаса сырья в каждом регионе и достаточного объема сырья для безубыточной разработки скважин можно выделить Регион_1 в отрицательную сторону, разница самая высокая, а привлекательным (пока что) кажется Регион_2.

3.3 Выберем скважины с максимальными значениями предсказаний. Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям. Рассчитаем прибыль для полученного объёма сырья.

In [28]:
#Напишу функцию расчета прибыли:
def profit_all(target, probabilities, count, profit, budget_200):
    sorted_probabilities = probabilities.sort_values(ascending=False)
    target_selected = target[sorted_probabilities.index][:count]
    return int(profit * target_selected.sum() - budget_200)

## Расчёт прибыли и рисков:

In [29]:
# метод для приведения к типу Series
def series_type(target, pred):
    target = target.reset_index(drop=True)
    pred = pd.Series(pred)
    return target, pred

In [30]:
# приведем целевые и прогнозные значения по всем регионам к одному типу
target_valid_0, predicted_valid_0 = series_type(target_valid_0, predicted_valid_0)
target_valid_1, predicted_valid_1 = series_type(target_valid_1, predicted_valid_1)
target_valid_2, predicted_valid_2 = series_type(target_valid_2, predicted_valid_2)

5.1. Применю технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли:

In [31]:
# метод получения основных расчётных параметров с помощью будстрепа
def bootstrap(target, predicted, points_200, profit_1, budget_200):
    state = RandomState(12345)
    values = []
    for i in range(1000):
        target_subsample = target.sample(n = points_500, replace=True, random_state=state)
        pred_subsumple = predicted[target_subsample.index]

        values.append(profit_all(target_subsample, pred_subsumple, points_200, profit_1, budget_200))
    values = pd.Series(values)
    values_mean = int(values.mean())
    lower = int(values.quantile(q=0.025))
    upper = int(values.quantile(q=0.975))
    risk = int(len(values[values < 0]) / len(values) * 100)
    return values_mean, lower, upper, risk

In [32]:
#Посчитаем основные параметры для трёх регионов
values_mean_0, lower_0, upper_0, risk_0  = bootstrap(target_valid_0, predicted_valid_0, points_200, profit_1, budget_200)
values_mean_1, lower_1, upper_1, risk_1  = bootstrap(target_valid_1, predicted_valid_1, points_200, profit_1, budget_200)
values_mean_2, lower_2, upper_2, risk_2  = bootstrap(target_valid_2, predicted_valid_2, points_200, profit_1, budget_200)

5.2. Найду среднюю прибыль, 95%-й доверительный интервал и риск убытков:

In [33]:
print('Средняя прибыль лучших месторождений региона_0 равна', values_mean_0)
print('Средняя прибыль лучших месторождений региона_1 равна', values_mean_1)
print('Средняя прибыль лучших месторождений региона_2 равна', values_mean_2)

Средняя прибыль лучших месторождений региона_0 равна 425938526
Средняя прибыль лучших месторождений региона_1 равна 515222772
Средняя прибыль лучших месторождений региона_2 равна 435008362


In [34]:
print('95% доверительный интервал для средней прибыли 200 лучших месторождений региона_0', lower_0, upper_0)
print('95% доверительный интервал для средней прибыли 200 лучших месторождений региона_1', lower_1, upper_1)
print('95% доверительный интервал для средней прибыли 200 лучших месторождений региона_2', lower_2, upper_2)

95% доверительный интервал для средней прибыли 200 лучших месторождений региона_0 -102090093 947976352
95% доверительный интервал для средней прибыли 200 лучших месторождений региона_1 68873225 931547590
95% доверительный интервал для средней прибыли 200 лучших месторождений региона_2 -128880546 969706953


In [35]:
print('Риск убытков региона_0 равен', risk_0, '%')
print('Риск убытков региона_1 равен', risk_1, '%')
print('Риск убытков региона_2 равен', risk_2, '%')

Риск убытков региона_0 равен 6 %
Риск убытков региона_1 равен 1 %
Риск убытков региона_2 равен 6 %


Вывод: Я загрузила данные, и все необходимые библиотеки. Изучида и подготовила данные, обучила модель линейной регресии, подготовила прогноз для скважин трех регионов. Средних запасов сырья было больше во во 2-м регионе, так же, как и RMSE было выше всего. 

Так же я сделала вывод, что достаточный объем сырья для безубыточной разработки новой скважиный равен 111.111 тыс. баррелей. 

Применила Bootstrap с 1000 выборок, для нахождения распределения прибыли, 95%-го доверительного интервала, риска убытков региона - для этого создала функцию, чтобы не дублировать код. 

После всей проведенной работы рекомендую Регион_1 для разработки скважин: так, как и средняя прибыль выше остальных (515 222 772 руб.), и 95% доверительный интервал лучше(68873225, 931547590), и риск убытков региона меньше(1%).